In [165]:
import pandas as pd
import folium
from IPython.core.display import display
import numpy as np
from folium.plugins import TimestampedGeoJson
from selenium import webdriver
from datetime import datetime


In [166]:
# IDEAS:
# for the world we could change color by city / country or use color densities (see links below in the code for implementation)

In [178]:
# USER SETTINGS:

input_csv_file = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSet/2019-09-14_Amsterdam_listings_detailed.csv'
# used later for display, can be a city or the word
airbnb_location = 'Amsterdam'
saving_path = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Outputs'

In [179]:
df = pd.read_csv(input_csv_file, low_memory = False);

movieData = df[['first_review', 'latitude', 'longitude']]
movieData = movieData.dropna()
movieData.first_review.asty = movieData.first_review.astype(str)
movieData['month'] = movieData.first_review.apply(lambda x: x.split('-')[1]).astype('int64')
movieData['year'] = movieData.first_review.apply(lambda x: x.split('-')[0]).astype('int64')
movieData = movieData.sort_values(by = 'year', ascending = True)

grouped = movieData.groupby(['year', 'month'])

In [ ]:
'''
for _, data in grouped:
    print('----->')
    print(data)
    print(type(data))
    print('LATITUDE')
    print(data.latitude)
    print('')
    for i in data:
        print(type(i))
'''

In [180]:
# inspiration: 
# https://towardsdatascience.com/visualizing-air-pollution-with-folium-maps-4ce1a1880677
# also look this:
# https://towardsdatascience.com/visualizing-bike-mobility-in-london-using-interactive-maps-for-absolute-beginners-3b9f55ccb59
# https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb#Timestamped-GeoJSON
# technical docs:
# https://python-visualization.github.io/folium/plugins.html
# https://python-visualization.github.io/folium/plugins.html
# cart styles:
# https://deparkes.co.uk/2016/06/10/folium-map-tiles/
# add like this in the code below: folium.Map(location = London, zoom_start = 12, tiles = "CartoDB dark_matter") 
# https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data

# use TimestampedGeoJson to animate the folium map. It requires a geojson input.
# create_geojson_features converts df into geojson

def create_geojson_features(groupedDF):
    print('> Creating GeoJSON features...')
    features = []
    for _, data in grouped:
        for _, row in data.iterrows():  
            date = str(row['year']) + '-' + str(row['month'])
            #date = datetime.date(year=row['year'], month=row['month'])
            #date = str(date.year) + '-' + str(date.month)
            # keep only date with date() (remove time) and only YYYY/MM (remove DD)
            date = datetime.strptime(date, '%Y-%m').date().__str__()[0:-3]
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type':'Point', 
                    'coordinates':[row['longitude'],row['latitude']]
                },
                'properties': {
                    # 'time': row['DatetimeBegin'].date().__str__(),
                    'time': date,
                    'icon': 'circle',
                    'iconstyle':{
                        'color': 'magenta',
                        'fillOpacity': 0.8,
                        'stroke': 'true',
                        'radius': 2
                    }
                }
            }
        features.append(feature)
    return features



def make_map(features, coord_start):
    print('> Making map...')
    airbnb_map = folium.Map(location = coord_start, control_scale = True, zoom_start = 11)

    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period = 'P1M'
        , add_last_point = True
        , auto_play = False
        , loop = False
        , max_speed = 5
        , loop_button = True
        , date_options = 'YYYY/MM'
        , time_slider_drag_update = True
    ).add_to(airbnb_map)
    print('> Done')
    return airbnb_map

def plot_airbnb(location, groupedDF, coord_start):
    print('Mapping {} AIRBNB'.format(location))
    # df = load_data(pollutant_ID)
    # df = clean_data(df)
    # df = prepare_data(df, pollutant_ID)
    features = create_geojson_features(groupedDF)
    return make_map(features, coord_start)

In [ ]:
'''
features = create_geojson_features(grouped)
coord_start = [movieData.longitude.mean(), movieData.latitude.mean()]
mm = make_map(features, coord_start)
'''

In [181]:
coord_start = [movieData.latitude.mean(), movieData.longitude.mean()]
airbnb_map = plot_airbnb(airbnb_location, grouped, coord_start)

# save map
saving_name = saving_path + '/' + airbnb_location + '.html'
airbnb_map.save(saving_name)

# display map
airbnb_map

Mapping Amsterdam AIRBNB
> Creating GeoJSON features...
> Making map...
> Done
